In [1]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import traceback
import types

In [28]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba_eyelash = Alibaba(lid, lpwd)
alibaba_eyelash.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [2]:
lid = 'glittertools@glittergroupcn.com'
lpwd = 'Songjianwei68970747'
alibaba_tools = Alibaba(lid, lpwd)
alibaba_tools.login()

In [10]:
alibaba_eyelash.browser.get('http://hz-productposting.alibaba.com/product/editing.htm?id=60732345735')
alibaba_eyelash.is_structured()

False

In [4]:
alibaba_tools.browser.get('https://post.alibaba.com/product/publish.htm?itemId=60684982220&pubType=similarPost')

In [19]:
browser = alibaba_tools.browser
attrs = {'name':'Top popular mechanical puller American style three jaw gear puller', 'keywords':[]}
attrs['keywords'].append('High Quality Three Jaw Gear Puller')
attrs['keywords'].append('Bearing Puller')
attrs['keywords'].append('Mechanical Puller')

input = browser.find_element_by_css_selector('#productTitle')
input.send_keys(Keys.CONTROL, 'a')
input.send_keys(Keys.BACKSPACE)
input.send_keys(attrs['name'])

input = browser.find_element_by_css_selector('#struct-productKeywords li:nth-child(1) input')
input.send_keys(Keys.CONTROL, 'a')
input.send_keys(attrs['keywords'][0])
input = browser.find_element_by_css_selector('#struct-productKeywords li:nth-child(2) input')
input.send_keys(Keys.CONTROL, 'a')
input.send_keys(attrs['keywords'][1])
input = browser.find_element_by_css_selector('#struct-productKeywords li:nth-child(3) input')
input.send_keys(Keys.CONTROL, 'a')
input.send_keys(attrs['keywords'][2])


In [17]:

input = browser.find_element_by_css_selector('#productTitle')

# input.send_keys(length*Keys.BACKSPACE)

In [48]:
def submit_product(self):
    is_structured = self.is_structured()
    # submit to next step
    btn_submit_next = None
    if is_structured:
        btn_submit_next = self.browser.find_element_by_css_selector('#struct-buttons button:nth-child(3)')
    else:
        btn_submit_next = self.browser.find_element_by_css_selector('button#submitFormNext')

    ActionChains(self.browser).move_to_element(btn_submit_next).perform()
    btn_submit_next.click()

    # 提交表格
    self.notify("primary", "提交产品")

    if is_structured:
        WebDriverWait(self.browser, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#button-prev')))
        btn_submit = self.browser.find_element_by_css_selector('#struct-buttons button:nth-child(3)')
    else:
        btn_submit = WebDriverWait(self.browser, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#submitFormBtnA')))

    self.click(btn_submit)

    WebDriverWait(self.browser, 30).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, '.next-step-item-last.next-step-item-process,.ui2-dialog-transition')))
    # 检查 是否 违规
    if self.browser.find_elements_by_css_selector('.ui2-dialog-transition'):
        words = ''
        for td in self.browser.find_elements_by_css_selector(
                '.ui2-dialog-transition tr:not(:first-child) td:nth-child(2)'):
            words = words + ' ' + td.text
        self.notify("danger", "出现 违规词 [ " + words + " ], 请手动确认，否则 30 秒后 提交 失败")
        WebDriverWait(self.browser, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.next-step-item-last.next-step-item-process')))

    self.notify("primary", "成功 提交 产品")
    
alibaba_eyelash.submit_product = types.MethodType(submit_product, alibaba_eyelash)

In [61]:
def update_product_price(self, data):
    if 'price' not in data:
        return False
    price = data['price']
    
    struct_fob = self.browser.find_element_by_css_selector('#struct-fob')
    ActionChains(self.browser).move_to_element(struct_fob).perform()
    
    if price['isTieredPricing']:
        self.notify("error", "该功能还没有实现 ... ...")
        return False
    else:
        self.notify("primary", "设置产品 [" + str(data['ali_id']) + "] 的价格区间 ... ...")
        
        time.sleep(0.5)
        input = self.browser.find_element_by_css_selector('#struct-fob div.icbu-fob-range .range-number-max input')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(str(price['price_range'][0]))
        
        time.sleep(0.5)
        input = self.browser.find_element_by_css_selector('#struct-fob div.icbu-fob-range .range-number-min input')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(str(price['price_range'][1]))
        return True
    
alibaba_eyelash.update_product_price = types.MethodType(update_product_price, alibaba_eyelash)

In [38]:
def update_product(self, data):
    browser = self.browser
    try:
        api = 'https://hz-productposting.alibaba.com/product/editing.htm?id='
        ali_id = str(data['ali_id'])
        if not browser:
            self.notify('danger', "没有登录，请先登录")
            return False

        self.notify("primary", "打开产品 [" + ali_id + "] 编辑页面 ... ...")
        browser.get(api + ali_id)
        # WebDriverWait(browser, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#editor-container')))

        result = False
        if self.update_product_price(data):
            result = True
        if self.update_product_detail_pictures(data):
            result = True

        if result:
            self.submit_product()
            
    except TimeoutException as e:
        self.notify('error', '更新产品 [' + ali_id + '] 数据 出错，超时，' + str(e))
        traceback.print_exc()
    except NoSuchElementException as e:
        self.notify('error', '更新产品 [' + ali_id + '] 数据 出错，没找到目标元素, ' + str(e))
        traceback.print_exc()
    except Exception as e:
        self.notify('error', '更新产品 [' + ali_id + '] 数据 出错, ' + str(e))
        traceback.print_exc()
    finally:
        return result
    
alibaba_eyelash.update_product = types.MethodType(update_product, alibaba_eyelash)

In [5]:
def get_posted_product_info(self, pn):
    browser = self.browser
    products = []
    try:
        self.notify("primary", '打开 产品管理 页面 ... ...')
        browser.get(self.api_product_manage)
        css_selector = '#ballon-container .list-item, #ballon-container .next-table-body tr'
        masker = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

        time.sleep(5)
        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()

        css_selector = '#ballon-container .manage-loading .next-loading-tip'
        div_tip = browser.find_elements_by_css_selector(css_selector)
        if len(div_tip) != 0:
            WebDriverWait(browser, 20).until(EC.staleness_of(div_tip[0]))

        msg = '切换 至 显示全部产品 ... ...'
        self.notify("primary", msg)
        css_selector = '#ballon-container div[role="tab"]:nth-child(1)'
        css_selector += ', #ballon-container .posting-manage-filter-row:first-child span:nth-child(2)'
        tab_all = WebDriverWait(browser, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
        self.click(tab_all)
        browser.implicitly_wait(1)
        css_selector = '#ballon-container .manage-loading .next-loading-tip'
        div_tip = browser.find_elements_by_css_selector(css_selector)
        if len(div_tip) != 0:
            WebDriverWait(browser, 20).until(EC.staleness_of(div_tip[0]))

        pn = int(pn)
        for counter in range(pn):
            msg = '查找 全部产品列表 第 ' + str(counter + 1) + ' 页'
            self.notify("primary", msg)

            html_source = browser.page_source
            doc = pq(html_source)
            items = doc('#ballon-container .list-item')
            if items:
                for item in items:
                    pq_item = pq(item)
                    product = {}
                    pq_a = pq_item.find('.product-subject a')
                    if len(pq_a) == 0:
                        continue
#                     print(pq_item.find('.product-model').text())
                    product['href'] = pq_a.attr('href')
                    product['title'] = pq_a.text().strip().lower()
                    product['pid'] = pq_item.find('.product-model').text().split(':')[1].strip()
                    product['category'] = pq_item.find('.group-name').text().split(':')[1].strip()
                    product['update'] = pq_item.find('.next-col:nth-child(5) span').text().strip()
                    product['price'] = pq_item.find('.next-col:nth-child(3)').text().strip()
                    product['quality'] = []
                    product['quality'].append(pq_item.find('.product-quality').text().strip())
                    pq_tags = pq_item.find('.product-tags .next-tag-body')
                    product['tags'] = []

                    for tag in pq_tags:
                        product['tags'].append(pq(tag).text().strip())

                    result = re.search('id=(\d+)$', product['href'])
                    if result:
                        ali_id = result.group(1)
                    else:
                        ali_id = re.search('_(\d+).htm', product['href']).group(1)
                    product['ali_id'] = ali_id
                    products.append(product)
            else:
                items = doc('#ballon-container .next-table-body tr')
                for item in items:
                    pq_item = pq(item)
                    product = {}
                    pq_a = pq_item.find('.product-subject a')
                    if len(pq_a) == 0:
                        continue

                    product['href'] = pq_a.attr('href')
                    product['title'] = pq_a.text().strip().lower()
                    product['pid'] = pq_item.find('.product-model').text().split(':')[1].strip()
                    product['category'] = pq_item.find('.product-group').text().split(':')[1].strip()
                    product['update'] = pq_item.find(
                        '.next-table-cell:nth-child(5)>div>div:first-child').text().strip()
                    product['price'] = pq_item.find('.next-table-cell:nth-child(3)').text().strip()
                    product['quality'] = []
                    product['quality'].append(pq_item.find('.product-quality').text().strip())
                    pq_tags = pq_item.find('.product-tags .next-tag-body')
                    product['tags'] = []

                    for tag in pq_tags:
                        product['tags'].append(pq(tag).text().strip())

                    result = re.search('id=(\d+)$', product['href'])
                    if result:
                        ali_id = result.group(1)
                    else:
                        ali_id = re.search('_(\d+).htm', product['href']).group(1)
                    product['ali_id'] = ali_id
                    products.append(product)

            if counter == pn - 1:
                break

            css_selector = '#ballon-container .next-pagination-pages .next'
            btn_next = browser.find_element_by_css_selector(css_selector)
            if btn_next.is_enabled():
                btn_next.click()
                browser.implicitly_wait(1)
                css_selector = '#ballon-container .manage-loading .next-loading-tip'
                div_tip = browser.find_elements_by_css_selector(css_selector)
                if len(div_tip) != 0:
                    WebDriverWait(browser, 20).until(EC.staleness_of(div_tip[0]))
            else:
                break

            self.notify("primary", "完成 查找！")
    except Exception as e:
        self.notify("danger", "爬取产品 数据 出错, " + str(e))
        traceback.print_exc()
        products = []
    finally:
        if self.socketio:
            self.notify("get_posted_product_info_result", products)
        return products

alibaba_eyelash.get_posted_product_info = types.MethodType(get_posted_product_info, alibaba_eyelash)
# alibaba_tools.get_posted_product_info = types.MethodType(get_posted_product_info, alibaba_tools)

In [29]:
ps = alibaba_eyelash.get_posted_product_info(99)
print(len(ps))

primary 打开 产品管理 页面 ... ...
primary 切换 至 显示全部产品 ... ...
primary 查找 全部产品列表 第 1 页
primary 完成 查找！
primary 查找 全部产品列表 第 2 页
primary 完成 查找！
primary 查找 全部产品列表 第 3 页
primary 完成 查找！
primary 查找 全部产品列表 第 4 页
primary 完成 查找！
primary 查找 全部产品列表 第 5 页
primary 完成 查找！
primary 查找 全部产品列表 第 6 页
primary 完成 查找！
primary 查找 全部产品列表 第 7 页
primary 完成 查找！
primary 查找 全部产品列表 第 8 页
primary 完成 查找！
primary 查找 全部产品列表 第 9 页
primary 完成 查找！
primary 查找 全部产品列表 第 10 页
primary 完成 查找！
primary 查找 全部产品列表 第 11 页
primary 完成 查找！
primary 查找 全部产品列表 第 12 页
primary 完成 查找！
primary 查找 全部产品列表 第 13 页
primary 完成 查找！
primary 查找 全部产品列表 第 14 页
primary 完成 查找！
primary 查找 全部产品列表 第 15 页
primary 完成 查找！
primary 查找 全部产品列表 第 16 页
primary 完成 查找！
primary 查找 全部产品列表 第 17 页
primary 完成 查找！
primary 查找 全部产品列表 第 18 页
primary 完成 查找！
primary 查找 全部产品列表 第 19 页
primary 完成 查找！
primary 查找 全部产品列表 第 20 页
primary 完成 查找！
primary 查找 全部产品列表 第 21 页
primary 完成 查找！
primary 查找 全部产品列表 第 22 页
primary 完成 查找！
primary 查找 全部产品列表 第 23 页
primary 完成 查找！
primary 查找 全部产品列表 第 24 页
primary 完

In [3]:
ps[0]

{'href': '//www.alibaba.com/product-detail/OEM-service-custom-eyelash-packaging-false_60769492342.html',
 'title': 'oem service custom eyelash packaging false eyelashes mink fur lashes 3d mink eyelashes',
 'pid': '11A49',
 'category': '3D Mink Eyelashes>11A Series',
 'update': '2018-09-03',
 'price': 'US $1.9 - 2.3 / Pair',
 'quality': ['精品'],
 'tags': [],
 'ali_id': '60769492342'}

In [33]:
processed = {}

In [2]:
%store -r ps

In [17]:
%store ps

Stored 'ps' (list)


In [13]:
from libs.CeleryTasks import tasks

In [34]:
data = {'price': {'isTieredPricing': False, 'price_range':[1.1, 1.4]}}
count = 0
t1 = time.time()
for p in ps:
    ali_id = p['ali_id']
    if ali_id in processed:
        continue
    if '1.1' in p['price']:
        continue
    if p['pid'].lower().startswith('22a'):
        data['ali_id'] = ali_id
        tasks.alibaba_update_product.apply_async(args=[data], queue='Eyelashes_pm')
        count += 1
        processed[ali_id] = p
        time.sleep(3)
#         if alibaba_eyelash.update_product(data):
#             count += 1
#             processed[ali_id] = p
t2 = time.time()
print(count)

70


In [70]:
print(str(round((t2-t1)/count, 2)), count)

11.17 244


In [49]:
'60761755703' in processed

False

In [44]:

alibaba_eyelash.is_structured()

True

In [21]:
count = 0
for p in ps:
    if p['pid'].lower().startswith('22a'):
        count += 1
count

129

In [ ]:
pictures = []
picture = {'old': "//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg"
pictures.append(picture)
picture = {'old': "//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg"}
picture['new'] = "//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg"
pictures.append(picture)
picture = {'old': "//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg"
pictures.append(picture)
price = {'isTieredPricing': False, 'price_range': [3.3, 4.8], 'tieredPricing': []}
data = dict(ali_id="60764140968", price=price, detail_pictures=pictures)

In [ ]:
alibaba.update_product(data)

In [ ]:
data

In [ ]:
element = browser.find_element_by_css_selector("#editor-container")
ActionChains(browser).move_to_element(element).perform()
# self.notify("primary", "开始 设置 产品详情页 ... ...")
# 点击 详情编辑器 ‘源代码’ 按钮， 清空内容，再切换回 富文本 编辑状态
browser.find_element_by_css_selector('#mceu_0 button').click()
textarea = WebDriverWait(browser, 5).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 textarea')))

In [ ]:
text = textarea.get_attribute('value')
soup = BeautifulSoup(text, 'html.parser')
for img in soup.find_all('img'):
    src = img['src']
    key = re.sub('//[^\.]*\.', '', src)
    
    for idx, pic in enumerate(pictures):
        if key not in pic['old']:
            continue
            
        img['src'] = pic['new']
        if img.has_attr('ori-width'):
            del img['ori-width']
        if img.has_attr('ori-height'):
            del img['ori-height']
        break


In [ ]:

js_templ = "document.querySelector('{selector}').value = '{value}';"
js = js_templ.format(selector='#mceu_46 textarea', value=re.sub('\n', '\\\\n', str(soup)))
browser.find_element_by_css_selector('#mceu_46 textarea').clear()
browser.execute_script(js)
browser.find_element_by_css_selector('#mceu_0 button').click()
WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 iframe')))